In [ ]:
training_videos_folder = ["/content/drive/MyDrive/dataset/Celeb-synthesis"]
# /content/drive/MyDrive/dataset/Celeb-synthesis
for folder in training_videos_folder:
    videos_path = glob.glob(join(folder, "*.mp4"))
    folder = "1"

    counter = 0
    for video_path in videos_path:
        cap = cv2.VideoCapture(video_path)
        vid = video_path.split("/")[-1]
        vid = vid.split(".")[0]
        frameRate = cap.get(5)  # frame rate

        if not exists("../train_frames/" + folder + "/video_" + str(int(counter))):
            makedirs("../train_frames/" + folder + "/video_" + str(int(counter)))

        while cap.isOpened():
            frameId = cap.get(1)  # current frame number
            ret, frame = cap.read()
            if not ret:
                break

            filename = ("../train_frames/"+ folder + "/video_"+ str(int(counter)) + "/image_"+ str(int(frameId) + 1)+ ".jpg")
            if frameId % 8 == 0:
              cv2.imwrite(filename, frame)

        cap.release()
        

In [ ]:
from facenet_pytorch import MTCNN
mtcnn = MTCNN(
    margin=40,
    select_largest=False,
    post_process=False
    # device="cuda:0"
)

In [ ]:
source_frames_folders = ["/train_frames/0/"]

dest_faces_folder = "../train_face/0/"

for i in source_frames_folders:
    counter = 0
    for j in listdir(i):
        imgs = glob.glob(join(i, j, "*.jpg"))
        if not exists(join(dest_faces_folder, j)):
            makedirs(join(dest_faces_folder, j))
        for img in imgs:
            frame = cv2.imread(img)
            frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            frame = Image.fromarray(frame)
            face = mtcnn(frame)

            try:
                imsave(
                    join(dest_faces_folder, j, k.split("/")[-1]),
                    face.permute(1, 2, 0).int().numpy(),
                )
            except AttributeError:
                print("Image un-abel to save hence skipping")
        counter += 1

# for training of framesh with labels equals 1;


# source_frames_folders = ["/train_frames/1/"]

# dest_faces_folder = "../train_face/1/"

# for i in source_frames_folders:
#     counter = 0
#     for j in listdir(i):
#         imgs = glob.glob(join(i, j, "*.jpg"))
#         if not exists(join(dest_faces_folder, j)):
#             makedirs(join(dest_faces_folder, j))
#         for img in imgs:
#             frame = cv2.imread(img)
#             frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
#             frame = Image.fromarray(frame)
#             face = mtcnn(frame)

#             try:
#                 imsave(
#                     join(dest_faces_folder, j, k.split("/")[-1]),
#                     face.permute(1, 2, 0).int().numpy(),
#                 )
#             except AttributeError:
#                 print("Image un-abel to save hence skipping")
#         counter += 1


In [ ]:
!zip -r /content/file.zip /train_face/0

In [ ]:
# Images are an easier way to represent the working model. In Machine Learning,

print("Welcome!")
frames_per_video = 47
img_size = 160

train_path = ["/content/drive/MyDrive/dataset/train_face/1", "/content/drive/MyDrive/dataset/train_face/0"]
c = 0
counter2 = 0
counter = 0
list_1 = [join(train_path[0], x) for x in listdir(train_path[0])]
list_0 = [join(train_path[1], x) for x in listdir(train_path[1])]

videoList = list_1 + list_0
shuffle(videoList)
train_data = []
train_label = []
images = []
labels = []


for i in videoList:
    img = glob.glob(join(i, "*.jpg"))
    img.sort(key=lambda f: int("".join(filter(str.isdigit, f))))
    images += img[: frames_per_video]
    label = [x.split("/")[-3] for x in img]
    labels += label[: frames_per_video]

    if counter % 10 == 0:
      print("Number of images feeded", counter)
    counter += 1


for j, k in zip(images, labels):

    img = cv2.imread(j)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = cv2.resize(
      img, (img_size, img_size), interpolation=cv2.INTER_AREA
        )
    train_data.append(img)
    train_label += [k]

    if counter2 % 10 == 0:
       print("Number of files done:", counter2)
    counter2 += 1
train_data = np.array(train_data)
train_label = np.array(train_label)
train_label = np_utils.to_categorical(train_label)

np.save("train_data.npy", train_data)
np.save("train_label.npy", train_label)